In [3]:
#구글 드라이브 연결
from google.colab import drive

drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [4]:
!pip install pycaret
!pip install optuna
!pip install jinja2
!pip install markupsafe==2.0.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 320 kB 7.6 MB/s 
     |████████████████████████████████| 262 kB 66.5 MB/s 
     |████████████████████████████████| 1.3 MB 72.7 MB/s 
     |████████████████████████████████| 3.3 MB 42.2 MB/s 
     |████████████████████████████████| 88 kB 9.2 MB/s 
     |████████████████████████████████| 6.8 MB 21.8 MB/s 
     |████████████████████████████████| 1.7 MB 46.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 167 kB 54.8 MB/s 
     |████████████████████████████████| 2.0 MB 51.3 MB/s 
     |████████████████████████████████| 17.0 MB 49.4 MB/s 
     |████████████████████████████████| 636 kB 71.7 MB/s 
     |████████████████████████████████| 10.4 MB 28.4 MB/s 
     |███████████████████████████████

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 308 kB 7.7 MB/s 
     |████████████████████████████████| 81 kB 10.0 MB/s 
     |████████████████████████████████| 112 kB 65.3 MB/s 
     |████████████████████████████████| 49 kB 7.5 MB/s 
     |████████████████████████████████| 147 kB 54.0 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=df18448192f39aba1f3b3614665242560bc2bb815730b54c16028e2fa33bdc71
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 2.1.1
    Uninstalling MarkupSafe-2.1.1:
     

In [5]:
import pandas as pd
import random
import os
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor

function ClickConnect(){ console.log("코랩 연결 끊김 방지"); document.querySelector("colab-toolbar-button#connect").click() } setInterval(ClickConnect, 60 * 1000)

In [6]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정

In [7]:
# 규칙에 나온 평가산식
from sklearn.metrics import mean_squared_error, make_scorer

# y_pred, y_val: dataframe 타입
def lg_nrmse(y_pred, y_val):
    # 각 Y Feature별 NRMSE 총합
    # Y_01 ~ Y_08 까지 20% 가중치 부여
    all_nrmse = []
    for idx in range(0,14): # ignore 'ID'
        rmse = mean_squared_error(y_val.iloc[:,idx], y_pred.iloc[:,idx], squared=False)
        nrmse = rmse/np.mean(np.abs(y_val.iloc[:,idx]))
        all_nrmse.append(nrmse)
    score = 1.2 * np.sum(all_nrmse[:7]) + 1.0 * np.sum(all_nrmse[7:14])
    return score

In [8]:
BASE_DATA = "/content/gdrive/MyDrive/LG/DATA/"

In [9]:
train_df = pd.read_csv(BASE_DATA+'train.csv')

train_x = train_df.filter(regex='X') # Input : X Featrue
train_y = train_df.filter(regex='Y') # Output : Y Feature

print(train_x.shape)
print(train_y.shape)

(39607, 56)
(39607, 14)


In [10]:
train_x.head(5)

,X_01,X_02,X_03,X_04,X_05,X_06,X_07,X_08,X_09,X_10,...,X_47,X_48,X_49,X_50,X_51,X_52,X_53,X_54,X_55,X_56
0,70.544,103.320,67.47,1,101.892,74.983,29.45,62.38,245.71,0.0,...,1,1,9706.03,137.043591,135.359219,147.837968,134.313475,125.605427,136.721425,125.028256
1,69.524,103.321,65.17,1,101.944,72.943,28.73,61.23,233.61,0.0,...,1,1,10423.43,133.736691,135.979817,149.924692,123.630583,127.893337,143.322659,124.877308
2,72.583,103.320,64.07,1,103.153,72.943,28.81,105.77,272.20,0.0,...,1,1,10948.53,132.805112,131.055355,146.814592,128.939070,127.012195,140.395688,122.238232
3,71.563,103.320,67.57,1,101.971,77.022,28.92,115.21,255.36,0.0,...,1,1,15007.03,134.138760,133.239422,139.720132,132.260824,130.723186,147.624829,134.875225
4,69.524,103.320,63.57,1,101.981,70.904,29.68,103.38,241.46,0.0,...,1,1,11051.03,142.728970,136.620022,134.853555,134.760252,125.647793,139.331105,123.272762


In [11]:
train_y.head(5)

,Y_01,Y_02,Y_03,Y_04,Y_05,Y_06,Y_07,Y_08,Y_09,Y_10,Y_11,Y_12,Y_13,Y_14
0,2.056,1.456,1.680,10.502,29.632,16.083,4.276,-25.381,-25.529,-22.769,23.792,-25.470,-25.409,-25.304
1,1.446,1.184,1.268,18.507,33.179,16.736,3.229,-26.619,-26.523,-22.574,24.691,-26.253,-26.497,-26.438
2,1.251,0.665,0.782,14.082,31.801,17.080,2.839,-26.238,-26.216,-22.169,24.649,-26.285,-26.215,-26.370
3,1.464,1.079,1.052,16.975,34.503,17.143,3.144,-25.426,-25.079,-21.765,24.913,-25.254,-25.021,-25.345
4,0.983,0.646,0.689,15.047,32.602,17.569,3.138,-25.376,-25.242,-21.072,25.299,-25.072,-25.195,-24.974


In [12]:
target_list = list(train_y.columns)
print(target_list)

['Y_01', 'Y_02', 'Y_03', 'Y_04', 'Y_05', 'Y_06', 'Y_07', 'Y_08', 'Y_09', 'Y_10', 'Y_11', 'Y_12', 'Y_13', 'Y_14']


In [13]:
submit = pd.read_csv(BASE_DATA+'sample_submission.csv')
submit.head(5)

,ID,Y_01,Y_02,Y_03,Y_04,Y_05,Y_06,Y_07,Y_08,Y_09,Y_10,Y_11,Y_12,Y_13,Y_14
0,TEST_00001,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,TEST_00002,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,TEST_00003,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,TEST_00004,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,TEST_00005,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
test_x = pd.read_csv(BASE_DATA+'/test.csv').drop(columns=['ID'])
test_x.head(5)

,X_01,X_02,X_03,X_04,X_05,X_06,X_07,X_08,X_09,X_10,...,X_47,X_48,X_49,X_50,X_51,X_52,X_53,X_54,X_55,X_56
0,68.504,103.321,76.67,1,101.867,73.963,30.51,63.57,239.80,0.0,...,1,1,17227.63,138.130429,129.460682,141.506570,133.427229,129.711498,133.138096,121.859684
1,67.485,103.320,69.37,1,101.992,67.845,28.03,116.99,189.23,0.0,...,1,1,17134.53,136.148839,128.266277,145.911745,131.196417,132.411480,133.629025,124.178623
2,69.524,103.320,68.97,1,101.884,77.022,29.65,205.68,214.93,0.0,...,1,1,14860.83,120.447446,119.988804,132.099908,120.450155,130.051708,128.252972,114.475628
3,69.524,103.320,65.87,1,101.866,73.963,28.15,103.38,180.80,0.0,...,1,1,15252.53,133.994695,125.069180,147.507669,123.142653,125.963665,139.666592,126.589253
4,73.603,103.321,66.67,1,101.891,74.983,29.92,71.20,231.93,0.0,...,1,1,10752.23,137.918202,135.116192,138.600473,127.173033,137.252712,134.411335,124.020016


In [15]:
# mkdir

import os
path = 'ensemble_model'
def makedirs(path): 
   try: 
        os.makedirs(path) 
   except OSError: 
       if not os.path.isdir(path): 
           raise
makedirs(path)

In [18]:
from pycaret.regression import * 
import time

SAVE = '/content/gdrive/MyDrive/'

model_list = []
predict_list = []
cnt = 0

for t in target_list:

    if os.path.exists(SAVE + f'best_model/{cnt+1}_model.pkl'):
      print(f"{cnt+1}_model exist")
      time.sleep(5)
      cnt+=1
      continue

    # train dataset 생성
    train = pd.concat([train_x,train_y[t]], axis =1)
    print(t + '학습 시작')
    reg = setup(data = train, target = t, session_id=42, silent=True)
    best= compare_models(sort = 'RMSE', n_select = 1)
    save_model(best, SAVE + f'best_model/{cnt+1}_model')
    
    
    cnt+=1
    


1_model exist
2_model exist
3_model exist
4_model exist
5_model exist
6_model exist
7_model exist
8_model exist
9_model exist
10_model exist
11_model exist
12_model exist
13_model exist
14_model exist


In [19]:
model_list

[]

In [20]:
predict_list

[]

In [21]:
submit.to_csv('./0803_pycaret.csv', index=False)

In [22]:
submit

,ID,Y_01,Y_02,Y_03,Y_04,Y_05,Y_06,Y_07,Y_08,Y_09,Y_10,Y_11,Y_12,Y_13,Y_14
0,TEST_00001,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,TEST_00002,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,TEST_00003,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,TEST_00004,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,TEST_00005,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39603,TEST_39604,0,0,0,0,0,0,0,0,0,0,0,0,0,0
39604,TEST_39605,0,0,0,0,0,0,0,0,0,0,0,0,0,0
39605,TEST_39606,0,0,0,0,0,0,0,0,0,0,0,0,0,0
39606,TEST_39607,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [23]:
# save model
for i, m in enumerate(model_list):
    save_model(m, f'{i+1}_model')

In [25]:
# 성능 확인
train_df = pd.read_csv(BASE_DATA + 'train.csv')

X_data = train_df.filter(regex='X') # Input : X Featrue
Y_data = train_df.filter(regex='Y') # Output : Y Feature

In [28]:
from sklearn.model_selection import train_test_split
from pycaret.regression import * 
X_train, X_test, Y_train, Y_test = train_test_split(X_data, Y_data, test_size=0.2, random_state=42)
predict_list = []
for i in range(0,14):
    loaded_model = load_model(f'/content/gdrive/MyDrive/best_model/{i+1}_model')
    print(loaded_model)
    # predict on new data
    new_data = X_test
    predict_label= predict_model(loaded_model, data = new_data).Label
    predict_list.append(predict_label)


INFO:logs:Initializing load_model()
INFO:logs:load_model(model_name=/content/gdrive/MyDrive/best_model/1_model, platform=None, authentication=None, verbose=True)


Transformation Pipeline and Model Successfully Loaded


INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=Pipeline(steps=[('dtypes',
                 DataTypes_Auto_infer(display_types=False,
                                      ml_usecase='regression', target='Y_01')),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_strategy='mean',
                                target_variable=None)),
                ('new_levels1',
                 New_Catagorical_Levels_in_TestData(replacement_strategy='least '
                                                                         '...
                ('binn', 'passthrough'), ('rem_outliers', 'passthrough'),
                ('cluster_all', 'passthrough'),
                ('dummy', Dummify(target='Y_01')),
                ('fix_perfect', Remove_100(target='Y_01')),
     

Pipeline(steps=[('dtypes',
                 DataTypes_Auto_infer(display_types=False,
                                      ml_usecase='regression', target='Y_01')),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_strategy='mean',
                                target_variable=None)),
                ('new_levels1',
                 New_Catagorical_Levels_in_TestData(replacement_strategy='least '
                                                                         '...
                ('binn', 'passthrough'), ('rem_outliers', 'passthrough'),
                ('cluster_all', 'passthrough'),
                ('dummy', Dummify(target='Y_01')),
                ('fix_perfect', Remove_100(target='Y_01')),
                ('clean_names', Clean_Colum_Names()),
                ('featur

INFO:logs:Initializing load_model()
INFO:logs:load_model(model_name=/content/gdrive/MyDrive/best_model/2_model, platform=None, authentication=None, verbose=True)


...

Pipeline(steps=[('dtypes',
                 DataTypes_Auto_infer(display_types=False,
                                      ml_usecase='regression', target='Y_13')),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_strategy='mean',
                                target_variable=None)),
                ('new_levels1',
                 New_Catagorical_Levels_in_TestData(replacement_strategy='least '
                                                                         '...
                ('binn', 'passthrough'), ('rem_outliers', 'passthrough'),
                ('cluster_all', 'passthrough'),
                ('dummy', Dummify(target='Y_13')),
                ('fix_perfect', Remove_100(target='Y_13')),
                ('clean_names', Clean_Colum_Names()),
                ('featur

INFO:logs:Initializing load_model()
INFO:logs:load_model(model_name=/content/gdrive/MyDrive/best_model/14_model, platform=None, authentication=None, verbose=True)


Transformation Pipeline and Model Successfully Loaded


INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=Pipeline(steps=[('dtypes',
                 DataTypes_Auto_infer(display_types=False,
                                      ml_usecase='regression', target='Y_14')),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_strategy='mean',
                                target_variable=None)),
                ('new_levels1',
                 New_Catagorical_Levels_in_TestData(replacement_strategy='least '
                                                                         '...
                ('binn', 'passthrough'), ('rem_outliers', 'passthrough'),
                ('cluster_all', 'passthrough'),
                ('dummy', Dummify(target='Y_14')),
                ('fix_perfect', Remove_100(target='Y_14')),
     

Pipeline(steps=[('dtypes',
                 DataTypes_Auto_infer(display_types=False,
                                      ml_usecase='regression', target='Y_14')),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_strategy='mean',
                                target_variable=None)),
                ('new_levels1',
                 New_Catagorical_Levels_in_TestData(replacement_strategy='least '
                                                                         '...
                ('binn', 'passthrough'), ('rem_outliers', 'passthrough'),
                ('cluster_all', 'passthrough'),
                ('dummy', Dummify(target='Y_14')),
                ('fix_perfect', Remove_100(target='Y_14')),
                ('clean_names', Clean_Colum_Names()),
                ('featur

In [29]:
print(predict_list)

[17963    1.275357
14237    1.346500
2992     1.338111
18587    1.274410
19112    1.454007
           ...   
33978    1.376999
22572    1.340688
2865     1.387345
22194    1.197629
37316    1.405035
Name: Label, Length: 7922, dtype: float64, 17963    1.006063
14237    1.053725
2992     1.071363
18587    1.032647
19112    1.095603
           ...   
33978    1.025419
22572    1.093634
2865     1.109630
22194    0.940096
37316    1.105107
Name: Label, Length: 7922, dtype: float64, 17963    0.954040
14237    0.983140
2992     1.004235
18587    0.935569
19112    1.080651
           ...   
33978    1.009310
22572    0.975867
2865     1.030175
22194    0.921805
37316    1.075960
Name: Label, Length: 7922, dtype: float64, 17963    13.43654
14237    13.88600
2992     14.03778
18587    13.52135
19112    13.74093
           ...   
33978    13.48031
22572    13.69286
2865     14.63673
22194    13.93363
37316    13.72079
Name: Label, Length: 7922, dtype: float64, 17963    30.802060
14237    31.4351

In [30]:
pred = pd.DataFrame(predict_list).T

pred.columns = Y_test.columns

pred

,Y_01,Y_02,Y_03,Y_04,Y_05,Y_06,Y_07,Y_08,Y_09,Y_10,Y_11,Y_12,Y_13,Y_14
17963,1.275357,1.006063,0.954040,13.43654,30.802060,16.476044,3.29035,-26.56215,-26.45716,-22.659737,24.169120,-26.43282,-26.46159,-26.47135
14237,1.346500,1.053725,0.983140,13.88600,31.435121,16.616855,3.04504,-25.92266,-26.10028,-22.262356,24.433383,-25.98403,-25.99697,-26.04725
2992,1.338111,1.071363,1.004235,14.03778,31.415341,16.954912,2.99790,-26.03693,-26.18886,-21.980580,24.396956,-26.08031,-25.99508,-26.10727
18587,1.274410,1.032647,0.935569,13.52135,31.113142,16.799133,3.23728,-26.32098,-26.47923,-22.382333,24.380110,-26.37395,-26.36023,-26.31831
19112,1.454007,1.095603,1.080651,13.74093,31.048698,16.576173,3.20979,-26.06878,-26.05978,-22.071935,24.274576,-25.90312,-25.90995,-25.90900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33978,1.376999,1.025419,1.009310,13.48031,31.093940,16.610743,3.21047,-26.27619,-26.35701,-22.471335,24.374246,-26.25188,-26.15048,-26.30190
22572,1.340688,1.093634,0.975867,13.69286,31.276584,16.221378,3.10480,-26.23626,-26.39104,-22.412507,24.232461,-26.25765,-26.29896,-26.30606
2865,1.387345,1.109630,1.030175,14.63673,31.795020,16.677552,2.95205,-26.13200,-26.10231,-22.163168,24.456180,-26.01404,-26.06847,-26.10251
22194,1.197629,0.940096,0.921805,13.93363,32.125331,16.659810,3.06431,-26.63835,-26.62392,-22.564953,24.525748,-26.50955,-26.50472,-26.50780


In [31]:
print(Y_test)

        Y_01   Y_02   Y_03    Y_04    Y_05    Y_06   Y_07    Y_08    Y_09  \
17963  1.326  0.824  0.783  13.284  31.265  17.226  3.664 -25.720 -25.642   
14237  1.069  0.737  0.660  13.152  31.978  17.011  3.168 -25.963 -25.848   
2992   1.278  1.146  0.974  13.604  32.053  16.582  3.153 -26.120 -25.842   
18587  1.476  1.037  1.049  15.259  32.896  17.680  2.976 -25.818 -25.606   
19112  1.428  1.083  1.038  17.379  32.569  16.785  3.079 -25.617 -25.460   
...      ...    ...    ...     ...     ...     ...    ...     ...     ...   
33978  1.374  1.051  0.709  17.020  34.488  16.682  3.248 -26.362 -26.338   
22572  1.376  0.907  1.118  10.528  29.808  15.943  3.192 -27.136 -27.555   
2865   1.720  1.155  1.124  15.023  29.330  17.696  2.952 -25.137 -25.199   
22194  1.212  0.731  0.509  15.485  30.060  17.004  3.547 -26.120 -26.015   
37316  1.876  1.369  1.644  16.084  33.581  17.518  3.059 -25.885 -25.887   

         Y_10    Y_11    Y_12    Y_13    Y_14  
17963 -22.282  25.073 -25.5

In [32]:
# 점수 확인
score = lg_nrmse(pred, Y_test)

print("pycaret score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

pycaret score: 1.9410 (0.0000)



In [34]:
predict_list = []
for i in range(0,14):
    loaded_model = load_model(f'/content/gdrive/MyDrive/best_model/{i+1}_model')
    print(loaded_model)
    # predict on new data
    new_data = test_x
    predict_label= predict_model(loaded_model, data = new_data).Label
    predict_list.append(predict_label)
    if i< 9:
        submit[f'Y_0{i+1}'] = predict_label
    else :
        submit[f'Y_{i+1}'] = predict_label

INFO:logs:Initializing load_model()
INFO:logs:load_model(model_name=/content/gdrive/MyDrive/best_model/1_model, platform=None, authentication=None, verbose=True)
INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=Pipeline(steps=[('dtypes',
                 DataTypes_Auto_infer(display_types=False,
                                      ml_usecase='regression', target='Y_01')),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_strategy='mean',
                                target_variable=None)),
                ('new_levels1',
                 New_Catagorical_Levels_in_TestData(replacement_strategy='least '
                                                                         '...
                ('binn', 'passthrough'), ('rem_outliers', 'passthrough'),
  

Transformation Pipeline and Model Successfully Loaded
Pipeline(steps=[('dtypes',
                 DataTypes_Auto_infer(display_types=False,
                                      ml_usecase='regression', target='Y_01')),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_strategy='mean',
                                target_variable=None)),
                ('new_levels1',
                 New_Catagorical_Levels_in_TestData(replacement_strategy='least '
                                                                         '...
                ('binn', 'passthrough'), ('rem_outliers', 'passthrough'),
                ('cluster_all', 'passthrough'),
                ('dummy', Dummify(target='Y_01')),
                ('fix_perfect', Remove_100(target='Y_01')),
                ('clean_

INFO:logs:Initializing load_model()
INFO:logs:load_model(model_name=/content/gdrive/MyDrive/best_model/2_model, platform=None, authentication=None, verbose=True)


Transformation Pipeline and Model Successfully Loaded
Pipeline(steps=[('dtypes',
                 DataTypes_Auto_infer(display_types=False,
                                      ml_usecase='regression', target='Y_02')),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_strategy='mean',
                                target_variable=None)),
                ('new_levels1',
                 New_Catagorical_Levels_in_TestData(replacement_strategy='least '
                                                                         '...
                ('binn', 'passthrough'), ('rem_outliers', 'passthrough'),
                ('cluster_all', 'passthrough'),
                ('dummy', Dummify(target='Y_02')),
                ('fix_perfect', Remove_100(target='Y_02')),
                ('clean_

INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=Pipeline(steps=[('dtypes',
                 DataTypes_Auto_infer(display_types=False,
                                      ml_usecase='regression', target='Y_02')),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_strategy='mean',
                                target_variable=None)),
                ('new_levels1',
                 New_Catagorical_Levels_in_TestData(replacement_strategy='least '
                                                                         '...
                ('binn', 'passthrough'), ('rem_outliers', 'passthrough'),
                ('cluster_all', 'passthrough'),
                ('dummy', Dummify(target='Y_02')),
                ('fix_perfect', Remove_100(target='Y_02')),
     

...

Pipeline(steps=[('dtypes',
                 DataTypes_Auto_infer(display_types=False,
                                      ml_usecase='regression', target='Y_13')),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_strategy='mean',
                                target_variable=None)),
                ('new_levels1',
                 New_Catagorical_Levels_in_TestData(replacement_strategy='least '
                                                                         '...
                ('binn', 'passthrough'), ('rem_outliers', 'passthrough'),
                ('cluster_all', 'passthrough'),
                ('dummy', Dummify(target='Y_13')),
                ('fix_perfect', Remove_100(target='Y_13')),
                ('clean_names', Clean_Colum_Names()),
                ('featur

INFO:logs:Initializing load_model()
INFO:logs:load_model(model_name=/content/gdrive/MyDrive/best_model/14_model, platform=None, authentication=None, verbose=True)


Transformation Pipeline and Model Successfully Loaded


INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=Pipeline(steps=[('dtypes',
                 DataTypes_Auto_infer(display_types=False,
                                      ml_usecase='regression', target='Y_14')),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_strategy='mean',
                                target_variable=None)),
                ('new_levels1',
                 New_Catagorical_Levels_in_TestData(replacement_strategy='least '
                                                                         '...
                ('binn', 'passthrough'), ('rem_outliers', 'passthrough'),
                ('cluster_all', 'passthrough'),
                ('dummy', Dummify(target='Y_14')),
                ('fix_perfect', Remove_100(target='Y_14')),
     

Pipeline(steps=[('dtypes',
                 DataTypes_Auto_infer(display_types=False,
                                      ml_usecase='regression', target='Y_14')),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_strategy='mean',
                                target_variable=None)),
                ('new_levels1',
                 New_Catagorical_Levels_in_TestData(replacement_strategy='least '
                                                                         '...
                ('binn', 'passthrough'), ('rem_outliers', 'passthrough'),
                ('cluster_all', 'passthrough'),
                ('dummy', Dummify(target='Y_14')),
                ('fix_perfect', Remove_100(target='Y_14')),
                ('clean_names', Clean_Colum_Names()),
                ('featur

In [35]:
submit

,ID,Y_01,Y_02,Y_03,Y_04,Y_05,Y_06,Y_07,Y_08,Y_09,Y_10,Y_11,Y_12,Y_13,Y_14
0,TEST_00001,1.398609,1.140097,1.096931,14.26939,31.263301,16.218683,3.23251,-26.13481,-26.20758,-22.101301,24.532215,-26.10783,-26.04005,-26.26911
1,TEST_00002,1.450727,1.157967,1.111629,13.76817,31.294836,16.684193,3.04410,-26.13409,-26.02013,-22.216489,24.504281,-25.99138,-26.10324,-26.11678
2,TEST_00003,1.424092,1.099312,1.040969,12.85079,31.663904,17.181059,3.18340,-25.99919,-26.15257,-22.288235,24.535453,-26.06369,-26.16543,-26.18087
3,TEST_00004,1.426172,1.121657,1.062394,14.24865,32.592541,17.127388,3.06191,-25.74397,-25.73432,-21.577524,24.907625,-25.69822,-25.74664,-25.70901
4,TEST_00005,1.278608,0.976507,0.910137,15.52136,31.692769,17.040039,3.18186,-25.90914,-25.67455,-21.637700,24.691644,-25.83871,-25.70400,-25.70967
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39603,TEST_39604,1.311397,0.998274,0.992276,12.37468,30.699830,16.741640,3.17707,-26.43989,-26.44765,-22.969935,24.332296,-26.48357,-26.43069,-26.45460
39604,TEST_39605,1.266081,0.924185,0.960545,13.94893,31.223275,16.698139,3.12474,-26.47847,-26.49647,-22.783472,24.432658,-26.36623,-26.39408,-26.50816
39605,TEST_39606,1.250434,0.951210,0.961247,12.75686,31.108801,16.641934,3.14417,-26.50774,-26.47061,-22.781161,24.291541,-26.35966,-26.45656,-26.48637
39606,TEST_39607,1.261561,0.940734,0.954000,13.51556,30.870001,16.726965,3.13943,-26.59450,-26.49093,-22.708910,24.509718,-26.44670,-26.43501,-26.47518


In [37]:
submit.to_csv('sub.csv')